We run a webshop that sells all kinds of products and where customers are able to write reviews about those items that they buy. We want to monitor the number of reviews by customers over time so that we can give incentives to do so in case these numbers diminish drastically.

The goal of this project is to compute summary statistics from tab separated files that contain reviews for Amazon products. The summary statistics should be stored in a database and displayed on a dashboard to see the evolution of the number of reviews per month.


In [1]:
from pyspark.sql import SparkSession
import mysql.connector

In [2]:

'''
Yeah, don't mind this. It is just for myself to check if I have the correct java version activated.

source ~/.bashrc
source ~/.bash_profile

uncomment the following line in /.bash_profile
 - export JAVA_HOME=$(/usr/libexec/java_home -v 1.8) pyspark
'''
import os
os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/jdk1.8.0_241.jdk/Contents/Home'

import subprocess
subprocess.check_output(['java', '-version'], stderr=subprocess.STDOUT)


b'java version "1.8.0_241"\nJava(TM) SE Runtime Environment (build 1.8.0_241-b07)\nJava HotSpot(TM) 64-Bit Server VM (build 25.241-b07, mixed mode)\n'

In [3]:
spark = SparkSession.builder.appName("Basics").getOrCreate()


In [4]:
amazon = spark.read.csv('amazon_reviews_multilingual_US_v1_00.tsv', sep='\t', header=True,inferSchema=True)

In [5]:
# amazon.show()

In [6]:
amazon = amazon.select('customer_id', 'review_date', 'product_category', 'star_rating')

In [7]:
# amazon.printSchema()

In [8]:
from pyspark.sql.functions import month
amazon = amazon.withColumn("month", month(amazon['review_date']))
amazon = amazon.drop('review_date')

In [9]:
# amazon.show()

In [10]:
feature_group = ['month','product_category','star_rating']
amazon = amazon.groupBy(feature_group).count()


In [11]:
# amazon.sort('product_category','month','star_rating').show()

In [ ]:
'''
Comment these lines if you don't want to get Rick rolled and learn the danger of running 
code without understanding what it does or reading the documentation.
'''

import webbrowser
webbrowser.open('https://www.youtube.com/watch?v=dQw4w9WgXcQ') 

In [12]:
# It is bad practice to hardcode your passwords into your programs. 
# That's why we prompt the user here to type it in himself every time the program runs
database_password = input("Can I have your password please?")

In [13]:
# Setting up a connection with our local database and creating a cursor

cnx = mysql.connector.connect(
    host = '127.0.0.1',
    user = 'root',
    passwd = database_password,
    auth_plugin='mysql_native_password',
    buffered=True
)

my_cursor = cnx.cursor()


In [14]:
# Here we run the "amazon.sql" file. For more information, see comments in that file.
path = "amazon.sql"
for line in open(path).read().split(';\n'):
    my_cursor.execute(line)

In [15]:
# Read the documentation to know what this method does you lazy fuck! 

collect = amazon.collect()

'''
Ok, I'll help because who in the world reads documentation: 
collect() returns all the records as a list of Row. 

So we'll get a list of all the records in this form: 
Row(month=8, product_category='Video', star_rating=1, count=286)
'''

"\nOk, I'll help because who in the world reads documentation: \ncollect() returns all the records as a list of Row. \n\nSo we'll get a list of all the records in this form: \nRow(month=8, product_category='Video', star_rating=1, count=286)\n"

In [16]:
total = amazon.count()

In [17]:
for x in range (total):
    
    #TODO: gebruik dict ipv int zodat volgorde niet meer uitmaakt
    month = collect[x][0]
    product_category = collect[x][1]
    star_rating = collect[x][2]
    count = collect[x][3]
    my_cursor.execute("INSERT INTO scores(month,product_category,star_rating,count) values(%s, %s,%s, %s)", (month,product_category,star_rating,count))


    if(x%10000 == 0):
        cnx.commit()
        
        progress = round(100 * x/total, 2)

        print("Progress: %s%%" % progress)
        
cnx.commit()
print("DONE!") 

Progress: 0.0%
DONE!
